# Coordinates

In [15]:
from math import pi, sqrt, sin, cos, tan

## Convert between WSG84 & UTM:

* utm:
    * Probably the simplest. Seems to do exactly what's needed.
    * https://github.com/Turbo87/utm
    * https://pypi.python.org/pypi/utm
    * This is the option selected!

* geographiclib
    * Solve a different problem. Though it could be an option with more work. Especially if 'cross' UTM region cases are a problem.
    * https://geographiclib.sourceforge.io/1.50/python/
    * https://pypi.org/project/geographiclib/

* pyproj:
    * ... Complicated ...
    * https://github.com/pyproj4/pyproj

* not using a librabry:
    * http://fr.wikipedia.org/wiki/Transverse_Universelle_de_Mercator 
    * the associated code is at the end of this notebook. 

### without external library

Let's consider we are in France and take the reference meridian phi0 = 7 degrees.

In [16]:
def wsg84_to_umt(lat, lon, lon0=None):
    """
    Convert from WSG84 coordinates in the geodesic system in DD format (e.g. (lat, lon) in degrees) to UTM coordinates (x, y) in metres.
    The reference meridian is given in lon0.
    """
    if lon0 is None:
        lon0 = float(int(lon))
    # convert from degree to radian
    lat = 2.*pi*lat/360.
    lon = 2.*pi*lon/360.
    lon0 = 2.*pi*lon0/360.
    # ref of the calculus below: http://fr.wikipedia.org/wiki/Transverse_Universelle_de_Mercator
    e = 0.0818192
    a = 6378.137
    v = 1. / sqrt(1. - e**2 * sin(lat)**2)
    A = (lon-lon0) * cos(lat)
    s = (1. - e**2/4. - 3*e**4/64. - 5*e**6/256.) * lat
    s -= (3*e**2/8. + 3*e**4/32. + 45*e**6/1024.) * sin(2.*lat)
    s += (15.*e**4/256. + 45*e**6/1024.) * sin(4.*lat)
    s -= (35*e**6/3072.) * sin(6*lat)
    T = tan(lat)**2
    C = e**2 / (1.-e**2) * cos(lat)**2
    k0 = 0.9996
    if(lat >= 0):
        N0 = 0.
    else:
        N0 = 10000
    E = 500. + k0*a*v*(A + (1.-T+C)*A**3/6. + (5-18*T+T**2)*A**5/120.)
    N = N0 + k0*a*(s+v*tan(lat)*(A**2/2+(5-T+9*C+4*C**2)*A**4/24+(61-58*T+T**2)*A**6/720))
    return (E*1000, N*1000)

In [17]:
wsg84_to_umt(43.63421388888889, 7.07895, 3.)

(829042.1752337032, 4839336.3133732965)

# Conversion from Degre Minute Second to Decimal

This might come in useful...

In [18]:
# Random place in the south of France
location = '''+43°38′3.17",+7°4′44.22"'''

Let's convert that to the DD notation:

In [19]:
import re
dms_re = re.compile("([+-])\s*(\d+)[^\d]+\s*(\d+)[^\d]+\s*([\d.]+)[^\d]+,\s*([+-])\s*(\d+)[^\d]+\s*(\d+)[^\d]+\s*([\d.]+)[^d]+")

def dms_to_dd(s):
    """Convert a string of the form +43°37′3.66",+7°4′43.08" (DMS notation) to a couple of float (DD notation)."""
    m = dms_re.search(s)
    latitude = float(m.group(2)) + float(m.group(3))/60 + float(m.group(4))/(60**2)
    if m.group(1) == '-':
        latitude = -latitude
    longitude = float(m.group(6)) + float(m.group(7))/60 + float(m.group(8))/(60**2)
    if m.group(5) == '-':
        longitude = - longitude
    return (latitude, longitude)


(lat, lon) = dms_to_dd(location)
(lat, lon)


(43.63421388888889, 7.07895)